In [1]:
import pandas as pd

import gc
####

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification
## felipe


from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA

nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_sm', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
!python -m spacy download es_core_news_sm

%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

2023-07-05 20:31:13.288599: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 62.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 21.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.0/425.0 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycorrector: filename=pycorrector-0.5.0-py3-none-any.whl size=4480931 sha256=52c41beebec5e322268a3052751e0f0777bf6b4a3e4ac2c448795dfbdb7b5900
  Stored in directory: /root/.cache/pip/wheels/03/cd/34/3c82eb07a21023abd3b8c334a96cb2b8f32cdd986d0f7d0b

# etl

In [2]:
import pandas as pd
import sqlite3

In [3]:


query1 = "SELECT NUMERO_PQR,DESCRIPCION_PQR FROM pqr_priorizar where MEDIO_RECEPCION !=  'VIRTUAL' and unico=1 and SINTOMA_PQR ='CONCEPTOS FACTURADOS' and FECHA_CREACION > '2023-06-21' and TECNOLOGIA ='FO'"

query2 = "(SELECT * FROM altas where ST_TIPO_SERVICIO='FTTH' AND FASE='ORDERF' AND (tipoTramite= 'VTA' OR tipoTramite=  'Sales')) AS myquery"

In [6]:

# Connect to the SQLite database file
conn = sqlite3.connect('/content/on2/pqr_priorizar.db')

# Read the SQL query into a pandas DataFrame

pqrs = pd.read_sql_query(query1, conn)

# Close the database connection
conn.close()

# Print the DataFrame
pqrs

#MDM-PQR-39191683

,NUMERO_PQR,DESCRIPCION_PQR
0,MDM-PQR-39499135,Nombre: MARIA ANGELICA RUBIANO ALDANA Cedula...
1,MDM-PQR-39502215,ERROR
2,MDM-PQR-39506135,Línea: 6012735526 Nombre de quien se comunica:...
3,MDM-PQR-39506189,Nombre: Oswaldo Hernandez Cedula: 79793694 Mov...
4,MDM-PQR-39506214,NOMBRE MARIA DILA GARZON RAMIREZ CC 2032794...
...,...,...
4214,MDM-PQR-39736985,NOMBRE: SAMIR MEJIA CC:79316942 CORREO:zamirt...
4215,MDM-PQR-39737146,envio de factura correcta ya que en extemprane...
4216,MDM-PQR-39737246,Cliente EDILIA TOSCANO CORDERO Cédula de Ciuda...
4217,MDM-PQR-39737284,NombreMARIA NELLY RODRIGUEZ ANDRADE Celular 3...


# TRASNFORMAR LAS DATOS DE TEXTO

In [7]:
sys.path.append('../librerias')

In [8]:
from clean_text import clean_text

entro2


In [9]:
pqrs = pqrs.fillna("vacios")



In [10]:
pqrs.isnull().sum()

NUMERO_PQR         0
DESCRIPCION_PQR    0
dtype: int64

In [11]:
#declare answers and questions
descripcion=pqrs.DESCRIPCION_PQR





#clean_descripcion=[k for k in clean_descripcion if 'soporte'  in k]
clean_descripcion = [clean_text(des) for des in descripcion]


len(clean_descripcion)

4219

In [21]:
clean_descripcion[190]

'    dire ion   b a   tramite cliente solicita descuento en su factura por falla masiva en este caso no se puede realizar ya que la factura la pago por ende el ajuste se realizara en su siguiente facturacion'

spelling

In [22]:
from tqdm import tqdm
from spelling import correct_sentence


hola


In [23]:
corrected_sentences = []
for text in tqdm(clean_descripcion, desc="Processing"):
    corrected_sentence = correct_sentence(text)
    corrected_sentences.append(corrected_sentence)

clean_descripcion=corrected_sentences

Processing: 100%|██████████| 4219/4219 [06:45<00:00, 10.40it/s]


lemma token stemm

In [24]:
from lemastem import data_preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


11659


In [25]:

clean_descripcion=data_preprocessing(clean_descripcion)
clean_descripcion[190]

['dirar',
 'tramitar',
 'solicitar',
 'descuento',
 'factura',
 'masivo',
 'caso',
 'poder',
 'realizar',
 'factura',
 'pago',
 'endar',
 'ajuste',
 'realizar',
 'siguiente',
 'facturacion']

In [26]:
clean_descripcion[0]

['unicar', 'solicitar', 'ajuste', 'factura', 'conto']

In [27]:
merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion'])


In [28]:
from unidecode import unidecode

def remove_accents(text):
    return unidecode(text)

# Aplicar la función a la columna 'Texto'
merged_df['descripcion'] = merged_df['descripcion'].apply(remove_accents)

In [30]:
merged_df

,descripcion
0,unicar solicitar ajuste factura conto
1,error
2,concepto facturar
3,explicar factura
4,robar cableado llevar desear saber facturacion
...,...
4214,ajuste factura fallo hacer
4215,parecer
4216,kdbvpp desear validar inactivo
4217,envio facturacion ajustado



clean_descripcion = [text for text in clean_descripcion if "svas" not in text and "directv" not in text  and "winsport" not in text and "hbo" not in text  ]

len(clean_descripcion)

In [31]:
from tfidf import TFIDF
import joblib

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from joblib import dump


X = merged_df['descripcion']
X_test=merged_df['descripcion']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_binario.joblib')

#pca = PCA(n_components=585)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 1517 features
train with old features:  (4219,)
train with new features: (4219, 537)


# cargar modelo

In [33]:
from joblib import dump

import joblib


model = joblib.load('/content/PRIORIZAR/modelo/modelo_binario.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.7202905940423128, bagging_freq=5,
               feature_fraction=0.7417324716447935,
               lambda_l1=0.21996878957588467, lambda_l2=0.05222507176000415,
               learning_rate=0.09500082144988906, min_child_samples=24,
               n_estimators=78, num_leaves=69)


In [34]:
y_pred = model.predict(X_train_new)

pqrs['prediccion_algoritmo'] = y_pred

pqrs['descripcion_clean']=merged_df['descripcion']



# Lista de palabras a filtrar
words_to_remove = ['svas', 'directv', 'paramont', 'hbo']

# Filtrar las filas del DataFrame
c_facturados_correcion = pqrs[~pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]

c_facturados_SVAS = pqrs[pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]



c_facturados_correcion


,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
0,MDM-PQR-39499135,Nombre: MARIA ANGELICA RUBIANO ALDANA Cedula...,0,unicar solicitar ajuste factura conto
1,MDM-PQR-39502215,ERROR,0,error
2,MDM-PQR-39506135,Línea: 6012735526 Nombre de quien se comunica:...,0,concepto facturar
3,MDM-PQR-39506189,Nombre: Oswaldo Hernandez Cedula: 79793694 Mov...,0,explicar factura
4,MDM-PQR-39506214,NOMBRE MARIA DILA GARZON RAMIREZ CC 2032794...,0,robar cableado llevar desear saber facturacion
...,...,...,...,...
4214,MDM-PQR-39736985,NOMBRE: SAMIR MEJIA CC:79316942 CORREO:zamirt...,1,ajuste factura fallo hacer
4215,MDM-PQR-39737146,envio de factura correcta ya que en extemprane...,0,parecer
4216,MDM-PQR-39737246,Cliente EDILIA TOSCANO CORDERO Cédula de Ciuda...,1,kdbvpp desear validar inactivo
4217,MDM-PQR-39737284,NombreMARIA NELLY RODRIGUEZ ANDRADE Celular 3...,1,envio facturacion ajustado


In [36]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = '/content/PRIORIZAR-1/SALIDAS'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename1 = f"{root_path}c_facturados_correcion_{current_date}.csv"
filename2 = f"{root_path}c_facturados_SVAS_{current_date}.csv"


c_facturados_correcion.to_csv(filename1, index=False)

c_facturados_SVAS.to_csv(filename2, index=False)




#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")

DONE


# modelo retipificacion

filter data with 0 label

In [40]:
retipificacion = c_facturados_correcion[c_facturados_correcion['prediccion_algoritmo'] == 0]
retipificacion

,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
0,MDM-PQR-39499135,Nombre: MARIA ANGELICA RUBIANO ALDANA Cedula...,0,unicar solicitar ajuste factura conto
2,MDM-PQR-39502215,ERROR,0,error
3,MDM-PQR-39505927,Apoyo WhatsApp // Línea: 6017228643 // Nombr...,0,unicar hacer cancelacion telefonico pagar fact...
7,MDM-PQR-39506189,Nombre: Oswaldo Hernandez Cedula: 79793694 Mov...,0,explicar factura
10,MDM-PQR-39506229,LINEA 6013573102 CC 79132221 NOMBRE DE QUIE...,0,unicar solicitar cuanto cobrar telefonia
...,...,...,...,...
5511,MDM-PQR-39643062,LINEA: 6013085818 NOMBRE: Daniel Corté...,0,liencer establecer
5512,MDM-PQR-39643074,MDM-PQR-39643074 Se comunica BERNARDO ARIAS AC...,0,
5515,MDM-PQR-39643138,informacion de factura ya que le informaron un...,0,costo decodificador explicar concepto otorgar ...
5517,MDM-PQR-39643273,nombre de quien se comunica: YANETH ALICIA URR...,0,unicar pedir envie factura


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split



X = retipificacion['descripcion_clean']
X_test=retipificacion['descripcion_clean']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_retipificacion.joblib')

#pca = PCA(n_components=361)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 1224 features
train with old features:  (2378,)
train with new features: (2378, 339)


In [47]:
model = joblib.load('/content/PRIORIZAR/modelo/retipificacion.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.8054451937682544, bagging_freq=3,
               feature_fraction=0.8508478559579478, lambda_l1=4.891566120846485,
               lambda_l2=0.010196403754110692,
               learning_rate=0.04089514301599394, min_child_samples=9,
               n_estimators=90, num_leaves=42)


In [48]:
y_pred = model.predict(X_train_new)

retipificacion["retipificacion_algoritmo"]=y_pred

<ipython-input-48-9505f33dc576>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retipificacion["retipificacion_algoritmo"]=y_pred


mapping = {'otros': 0, 'INCREMENTO DE TARIFA': 1, 'ASEGURAMIENTO': 2,  'BENEFICIO DE RETENCION': 3}


In [49]:
retipificacion

,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean,retipificacion_algoritmo
0,MDM-PQR-39499135,Nombre: MARIA ANGELICA RUBIANO ALDANA Cedula...,0,unicar solicitar ajuste factura conto,1
2,MDM-PQR-39502215,ERROR,0,error,1
3,MDM-PQR-39505927,Apoyo WhatsApp // Línea: 6017228643 // Nombr...,0,unicar hacer cancelacion telefonico pagar fact...,1
7,MDM-PQR-39506189,Nombre: Oswaldo Hernandez Cedula: 79793694 Mov...,0,explicar factura,1
10,MDM-PQR-39506229,LINEA 6013573102 CC 79132221 NOMBRE DE QUIE...,0,unicar solicitar cuanto cobrar telefonia,0
...,...,...,...,...,...
5511,MDM-PQR-39643062,LINEA: 6013085818 NOMBRE: Daniel Corté...,0,liencer establecer,0
5512,MDM-PQR-39643074,MDM-PQR-39643074 Se comunica BERNARDO ARIAS AC...,0,,0
5515,MDM-PQR-39643138,informacion de factura ya que le informaron un...,0,costo decodificador explicar concepto otorgar ...,0
5517,MDM-PQR-39643273,nombre de quien se comunica: YANETH ALICIA URR...,0,unicar pedir envie factura,0


In [50]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = '/content/PRIORIZAR/SALIDAS/'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename3 = f"{root_path}c_facturados_retipificacion_{current_date}.csv"



retipificacion.to_csv(filename3, index=False)





#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")

DONE
